In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/jira_/Desktop/new_stockdata.csv')
d = df.values
ntotal=len(df)


d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 17
n_encoder_h_2 = 13
n_encoder_h_3 = 9


#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 13
n_decoder_h_2 = 17
n_decoder_h_3 = 22



# Parameters
learning_rate = 0.01

#batch_size = 7
display_step = 1

total_batch=6

training_epochs = 7000
#we store the variables here



In [2]:
len(trainset[1])

204

In [3]:
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [22, n_encoder_h_1], [n_encoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_encoder_h_1, n_encoder_h_2], [n_encoder_h_2], phase_train)

       
        with tf.variable_scope("code"):
            output = layer(h_2, [n_encoder_h_2, n_code], [n_code], phase_train)

    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [n_code, n_decoder_h_1], [n_decoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_decoder_h_1, n_decoder_h_2], [n_decoder_h_2], phase_train)


        with tf.variable_scope("output"):
            output = layer(h_2, [n_decoder_h_2,22], [22], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '9'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 22]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive100=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/jira_/Desktop/tmp/model.ckpt')
                        togive4=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,22)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive4.append(output_any_image)
                            
                        togive100=togive4
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)


we begin
Epoch: 0001 cost = 2.468343774
Validation Loss: 2.5518234
Epoch: 0002 cost = 2.445704579
Validation Loss: 2.4591238
Epoch: 0003 cost = 2.420328418
Validation Loss: 2.3168895
Epoch: 0004 cost = 2.393536369
Validation Loss: 2.0917156
Epoch: 0005 cost = 2.364512960
Validation Loss: 2.0093427
Epoch: 0006 cost = 2.334531267
Validation Loss: 1.9537092
Epoch: 0007 cost = 2.305011630
Validation Loss: 1.9325299
Epoch: 0008 cost = 2.279831449
Validation Loss: 1.9459774
Epoch: 0009 cost = 2.259823322
Validation Loss: 2.116188
Epoch: 0010 cost = 2.242426912
Validation Loss: 2.212453
Epoch: 0011 cost = 2.224889636
Validation Loss: 2.2083535
Epoch: 0012 cost = 2.204747677
Validation Loss: 2.2020564
Epoch: 0013 cost = 2.183557987
Validation Loss: 2.2065935
Epoch: 0014 cost = 2.167590300
Validation Loss: 2.2503676
Epoch: 0015 cost = 2.156120380
Validation Loss: 2.2921655
Epoch: 0016 cost = 2.145628572
Validation Loss: 2.3295362
Epoch: 0017 cost = 2.135478417
Validation Loss: 2.4308743
Epoch: 

Validation Loss: 1.1723386
Epoch: 0143 cost = 1.183971842
Validation Loss: 1.3634769
Epoch: 0144 cost = 1.179052969
Validation Loss: 1.4090725
Epoch: 0145 cost = 1.174162169
Validation Loss: 1.3556831
Epoch: 0146 cost = 1.169299304
Validation Loss: 1.2583245
Epoch: 0147 cost = 1.164464275
Validation Loss: 1.4083359
Epoch: 0148 cost = 1.159656604
Validation Loss: 1.2408531
Epoch: 0149 cost = 1.154876391
Validation Loss: 1.0703712
Epoch: 0150 cost = 1.150123636
Validation Loss: 1.130848
Epoch: 0151 cost = 1.145397961
Validation Loss: 1.2396184
Epoch: 0152 cost = 1.140699228
Validation Loss: 1.3373202
Epoch: 0153 cost = 1.136027435
Validation Loss: 1.1964624
Epoch: 0154 cost = 1.131382287
Validation Loss: 1.1892837
Epoch: 0155 cost = 1.126763980
Validation Loss: 1.1530842
Epoch: 0156 cost = 1.122172097
Validation Loss: 1.1897205
Epoch: 0157 cost = 1.117606640
Validation Loss: 1.1928749
Epoch: 0158 cost = 1.113067547
Validation Loss: 1.2131987
Epoch: 0159 cost = 1.108554343
Validation Loss

Epoch: 0285 cost = 0.706056009
Validation Loss: 0.79322594
Epoch: 0286 cost = 0.703873008
Validation Loss: 0.8041287
Epoch: 0287 cost = 0.701703499
Validation Loss: 0.76072943
Epoch: 0288 cost = 0.699552447
Validation Loss: 0.73094314
Epoch: 0289 cost = 0.697412342
Validation Loss: 0.72945404
Epoch: 0290 cost = 0.695279022
Validation Loss: 0.7366519
Epoch: 0291 cost = 0.693158954
Validation Loss: 0.7501501
Epoch: 0292 cost = 0.691054920
Validation Loss: 0.74071145
Epoch: 0293 cost = 0.688961983
Validation Loss: 0.7176694
Epoch: 0294 cost = 0.686878880
Validation Loss: 0.81206626
Epoch: 0295 cost = 0.684810261
Validation Loss: 0.8234612
Epoch: 0296 cost = 0.682758192
Validation Loss: 0.773479
Epoch: 0297 cost = 0.680720677
Validation Loss: 0.72779393
Epoch: 0298 cost = 0.678698440
Validation Loss: 0.7355823
Epoch: 0299 cost = 0.676695009
Validation Loss: 0.76081496
Epoch: 0300 cost = 0.674711843
Validation Loss: 0.7588526
Epoch: 0301 cost = 0.672747960
Validation Loss: 0.7659529
Epoch: 

Epoch: 0427 cost = 0.488533402
Validation Loss: 0.5765185
Epoch: 0428 cost = 0.487625390
Validation Loss: 0.5618245
Epoch: 0429 cost = 0.486603027
Validation Loss: 0.54557073
Epoch: 0430 cost = 0.485408649
Validation Loss: 0.5313303
Epoch: 0431 cost = 0.484306415
Validation Loss: 0.5149778
Epoch: 0432 cost = 0.483449568
Validation Loss: 0.602416
Epoch: 0433 cost = 0.482548460
Validation Loss: 0.5695739
Epoch: 0434 cost = 0.481335973
Validation Loss: 0.53311914
Epoch: 0435 cost = 0.480208039
Validation Loss: 0.52901083
Epoch: 0436 cost = 0.479245822
Validation Loss: 0.51908284
Epoch: 0437 cost = 0.478221714
Validation Loss: 0.5223415
Epoch: 0438 cost = 0.477170562
Validation Loss: 0.5298717
Epoch: 0439 cost = 0.476187209
Validation Loss: 0.51411843
Epoch: 0440 cost = 0.475237285
Validation Loss: 0.5706661
Epoch: 0441 cost = 0.474244833
Validation Loss: 0.55205566
Epoch: 0442 cost = 0.473231827
Validation Loss: 0.52795684
Epoch: 0443 cost = 0.472258886
Validation Loss: 0.52501845
Epoch: 

Epoch: 0567 cost = 0.381308402
Validation Loss: 0.4540323
Epoch: 0568 cost = 0.380799428
Validation Loss: 0.43182275
Epoch: 0569 cost = 0.380263428
Validation Loss: 0.4076757
Epoch: 0570 cost = 0.379693146
Validation Loss: 0.4331367
Epoch: 0571 cost = 0.379130185
Validation Loss: 0.4361411
Epoch: 0572 cost = 0.378617540
Validation Loss: 0.42152604
Epoch: 0573 cost = 0.378141120
Validation Loss: 0.42440426
Epoch: 0574 cost = 0.377637843
Validation Loss: 0.4226342
Epoch: 0575 cost = 0.377078911
Validation Loss: 0.4358241
Epoch: 0576 cost = 0.376513397
Validation Loss: 0.42562267
Epoch: 0577 cost = 0.375991096
Validation Loss: 0.41754425
Epoch: 0578 cost = 0.375508030
Validation Loss: 0.41199
Epoch: 0579 cost = 0.375024959
Validation Loss: 0.41195384
Epoch: 0580 cost = 0.374501149
Validation Loss: 0.41073325
Epoch: 0581 cost = 0.373946249
Validation Loss: 0.4141804
Epoch: 0582 cost = 0.373420174
Validation Loss: 0.42205542
Epoch: 0583 cost = 0.372953986
Validation Loss: 0.42629814
Epoch: 

...

Validation Loss: 0.038268995
Epoch: 6310 cost = 0.026708957
Validation Loss: 0.03875612
Epoch: 6311 cost = 0.026715943
Validation Loss: 0.041234225
Epoch: 6312 cost = 0.026752239
Validation Loss: 0.039923266
Epoch: 6313 cost = 0.026789777
Validation Loss: 0.03724534
Epoch: 6314 cost = 0.026794315
Validation Loss: 0.04037678
Epoch: 6315 cost = 0.026778380
Validation Loss: 0.041175213
Epoch: 6316 cost = 0.026760708
Validation Loss: 0.0390853
Epoch: 6317 cost = 0.026750563
Validation Loss: 0.035699435
Epoch: 6318 cost = 0.026740858
Validation Loss: 0.036211036
Epoch: 6319 cost = 0.026722884
Validation Loss: 0.033492543
Epoch: 6320 cost = 0.026704818
Validation Loss: 0.033029445
Epoch: 6321 cost = 0.026701067
Validation Loss: 0.033415597
Epoch: 6322 cost = 0.026721306
Validation Loss: 0.034358814
Epoch: 6323 cost = 0.026756775
Validation Loss: 0.03418846
Epoch: 6324 cost = 0.026778569
Validation Loss: 0.03391096
Epoch: 6325 cost = 0.026775791
Validation Loss: 0.033326536
Epoch: 6326 cost =

Epoch: 6448 cost = 0.026733295
Validation Loss: 0.034597844
Epoch: 6449 cost = 0.026766423
Validation Loss: 0.0337943
Epoch: 6450 cost = 0.026781124
Validation Loss: 0.033309303
Epoch: 6451 cost = 0.026783608
Validation Loss: 0.035045393
Epoch: 6452 cost = 0.026764248
Validation Loss: 0.03596205
Epoch: 6453 cost = 0.026725470
Validation Loss: 0.034964453
Epoch: 6454 cost = 0.026686451
Validation Loss: 0.0348
Epoch: 6455 cost = 0.026667681
Validation Loss: 0.034581553
Epoch: 6456 cost = 0.026680977
Validation Loss: 0.034746606
Epoch: 6457 cost = 0.026709789
Validation Loss: 0.03567064
Epoch: 6458 cost = 0.026741055
Validation Loss: 0.033526644
Epoch: 6459 cost = 0.026762206
Validation Loss: 0.035954986
Epoch: 6460 cost = 0.026768988
Validation Loss: 0.039896272
Epoch: 6461 cost = 0.026755041
Validation Loss: 0.038108077
Epoch: 6462 cost = 0.026722085
Validation Loss: 0.037619624
Epoch: 6463 cost = 0.026684220
Validation Loss: 0.035651673
Epoch: 6464 cost = 0.026661796
Validation Loss: 0

Epoch: 6598 cost = 0.026692286
Validation Loss: 0.034828182
Epoch: 6599 cost = 0.026702242
Validation Loss: 0.03368323
Epoch: 6600 cost = 0.026693479
Validation Loss: 0.03743658
Epoch: 6601 cost = 0.026674758
Validation Loss: 0.0365251
Epoch: 6602 cost = 0.026650370
Validation Loss: 0.03686373
Epoch: 6603 cost = 0.026642036
Validation Loss: 0.03634675
Epoch: 6604 cost = 0.026648203
Validation Loss: 0.034745723
Epoch: 6605 cost = 0.026654278
Validation Loss: 0.03731191
Epoch: 6606 cost = 0.026660950
Validation Loss: 0.04140692
Epoch: 6607 cost = 0.026677991
Validation Loss: 0.0389506
Epoch: 6608 cost = 0.026707956
Validation Loss: 0.036644123
Epoch: 6609 cost = 0.026732816
Validation Loss: 0.034304008
Epoch: 6610 cost = 0.026730518
Validation Loss: 0.034400117
Epoch: 6611 cost = 0.026711651
Validation Loss: 0.03628972
Epoch: 6612 cost = 0.026693879
Validation Loss: 0.03649828
Epoch: 6613 cost = 0.026682703
Validation Loss: 0.035369255
Epoch: 6614 cost = 0.026674027
Validation Loss: 0.04

Validation Loss: 0.031950854
Epoch: 6746 cost = 0.026620988
Validation Loss: 0.033719365
Epoch: 6747 cost = 0.026649763
Validation Loss: 0.035121903
Epoch: 6748 cost = 0.026703362
Validation Loss: 0.034776878
Epoch: 6749 cost = 0.026725306
Validation Loss: 0.033645973
Epoch: 6750 cost = 0.026711376
Validation Loss: 0.035780363
Epoch: 6751 cost = 0.026676856
Validation Loss: 0.034679852
Epoch: 6752 cost = 0.026635464
Validation Loss: 0.03394659
Epoch: 6753 cost = 0.026613965
Validation Loss: 0.036137063
Epoch: 6754 cost = 0.026605642
Validation Loss: 0.036612134
Epoch: 6755 cost = 0.026602543
Validation Loss: 0.036507286
Epoch: 6756 cost = 0.026619217
Validation Loss: 0.036518782
Epoch: 6757 cost = 0.026667373
Validation Loss: 0.03701601
Epoch: 6758 cost = 0.026700752
Validation Loss: 0.036760673
Epoch: 6759 cost = 0.026695543
Validation Loss: 0.038160738
Epoch: 6760 cost = 0.026671432
Validation Loss: 0.035935037
Epoch: 6761 cost = 0.026638059
Validation Loss: 0.03389677
Epoch: 6762 co

Validation Loss: 0.034575332
Epoch: 6894 cost = 0.026599983
Validation Loss: 0.03466806
Epoch: 6895 cost = 0.026578245
Validation Loss: 0.03536304
Epoch: 6896 cost = 0.026569267
Validation Loss: 0.033227578
Epoch: 6897 cost = 0.026560711
Validation Loss: 0.03319793
Epoch: 6898 cost = 0.026562841
Validation Loss: 0.032882936
Epoch: 6899 cost = 0.026595451
Validation Loss: 0.034758553
Epoch: 6900 cost = 0.026638497
Validation Loss: 0.034164477
Epoch: 6901 cost = 0.026648787
Validation Loss: 0.035266865
Epoch: 6902 cost = 0.026633242
Validation Loss: 0.033490833
Epoch: 6903 cost = 0.026605823
Validation Loss: 0.03353441
Epoch: 6904 cost = 0.026584404
Validation Loss: 0.034659237
Epoch: 6905 cost = 0.026573487
Validation Loss: 0.036698546
Epoch: 6906 cost = 0.026560474
Validation Loss: 0.037173785
Epoch: 6907 cost = 0.026548958
Validation Loss: 0.03995379
Epoch: 6908 cost = 0.026556495
Validation Loss: 0.04310875
Epoch: 6909 cost = 0.026588818
Validation Loss: 0.044186067
Epoch: 6910 cost 